In [1]:
import numpy as np
import matplotlib.pyplot as plt
import openpyxl
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# Decision Trees Implementation


This code is used to predict voting habits of responsees using the following factors:


    -Urban or rural primary sample unit
    -Views on countries current economic situation
    -If they discuss politics
    -Requests government action
    -Views on corruption
    -Education
    

# Binarisation:

## Urban/Rural:

&emsp;Urban: 0

&emsp;Rural: 1

&emsp;Tribal: 2

&emsp;Rural farm = 3


## Views on country's economic condition:


&emsp;Very Bad:0

&emsp;Fairly Bad: 1

&emsp;Fairly Good: 2

&emsp;Very Good: 3

&emsp;Neither good nor bad: 4

&emsp;Don't know: -1



## Discuss Politics:

&emsp;Never: 0

&emsp;Occasionally: 1

&emsp;Frequently: 2

&emsp;Don't know: -1



## Voted:

&emsp;Did Voted: 1

&emsp;Don't know: -1

&emsp;All others: 0


## Requests gov action

&emsp;No, would never do this: 0

&emsp;No, but would do if had the chance: 1

&emsp;Yes, once or twice: 2

&emsp;Yes, several times: 3 

&emsp;Yes, often: 4

&emsp;Don't know: -1


## Corruption:

&emsp;Decreased a lot: 0

&emsp;Decreased somewhat: 1

&emsp;Stayed the same: 2

&emsp;Increased somewhat: 3

&emsp;Increased a lot: 4

&emsp;Don't know: -1



## Education:

&emsp; No formal schooling: 0

&emsp;Informal schooling only: 1

&emsp;Some primary schooling: 2

&emsp;Primary school completed, Some secondary school / high school: 3

&emsp;Secondary school / high school completed: 4

&emsp;Some university: 5

&emsp;University completed, Post-graduate, Post-secondary qualifications, other than university: 6

&emsp;Don't know, Inside the house, Outside the compound: -1





&emsp;

## Getting and processing the data

The Data is stored in an excel file called "MLData_Trimmed2.xlsx". Using opnpyxl, the data from the file is loaded into the variable "sheet", and then into an $2389x8$ array

In [2]:
xlsx_file = Path('MLData_Trimmed2.xlsx')
wb_obj = openpyxl.load_workbook(xlsx_file)
sheet = wb_obj.active

In [5]:
data = np.zeros((sheet.max_row-2, sheet.max_column))

ir = 0
ic = 0

#Iterates trhough each row/cell and places data in $data
for row in sheet.iter_rows(2, max_row = sheet.max_row - 1):
    for cell in row:
        if ic == 0:
            #RESPNO
            data[ir, ic] = cell.value
            
        elif ic == 1:
            #Urban/Rural
            if cell.value == 'Urban':
                data[ir, ic] = 0
            elif cell.value == 'Rural':
                data[ir, ic] = 1
            elif cell.value == 'Tribal':
                data[ir, ic] = 2
            else:
                data[ir, ic] = 3
                
        elif ic == 2:
            #Views on Country's Economic Condition
            if cell.value == "Don't know":
                data[ir, ic] = -1
            elif cell.value == 'Very bad':
                data[ir, ic] = 0
            elif cell.value == 'Fairly bad':
                data[ir, ic] = 1
            elif cell.value == 'Fairly Good':
                data[ir, ic] = 2
            elif cell.value == 'Very good':
                data[ir, ic] = 3
            elif cell.value == 'Neither good nor bad':
                data[ir, ic] = 4
                
        elif ic == 3:
            #Discuss Politics?
            if cell.value == "Don't know":
                data[ir, ic] = -1
            elif cell.value == "Never":
                data[ir, ic] = 0
            elif cell.value == "Occasionally":
                data[ir, ic] = 1
            elif cell.value == "Frequently":
                data[ir, ic] = 2
                
        elif ic == 4:
            #Voted?
            if cell.value == "Don't know":
                data[ir, ic] = -1
            elif cell.value == "You voted in the elections":
                data[ir, ic] = 1
            else:
                data[ir, ic] = 0
            
        elif ic == 5:
            #Requests gov assistance
            if cell.value == "Don't know":
                data[ir, ic] = -1
            elif cell.value == "No, would never do this":
                data[ir, ic] = 0
            elif cell.value == "No, but would do if had the chance":
                data[ir, ic] = 1
            elif cell.value == "Yes, once or twice":
                data[ir, ic] = 2
            elif cell.value == "Yes, several times":
                data[ir, ic] = 3
            elif cell.value == "Yes, often":
                data[ir, ic] = 4
            
            
        elif ic == 6:
            #Corruption
            if cell.value == "Don't know":
                data[ir, ic] = -1
            elif cell.value == "Decreased a lot":
                data[ir, ic] = 0
            elif cell.value == "Decreased somewhat":
                data[ir, ic] = 1
            elif cell.value == "Stayed the same":
                data[ir, ic] = 2
            elif cell.value == "Increased somewhat":
                data[ir, ic] = 3
            elif cell.value == "Increased a lot":
                data[ir, ic] = 4
        elif ic == 7:
            #Education
            invalidStrings = ["Don't know", "Outside the compound", "Inside the house"]
            if cell.value in invalidStrings:
                data[ir, ic] = -1
            elif cell.value == "No formal schooling":
                data[ir, ic] = 0
            elif cell.value == "Informal schooling only":
                data[ir, ic] = 1
            elif cell.value == "Some primary schooling":
                data[ir, ic] = 2
            elif cell.value == "Primary school completed" or cell.value == "Some secondary school / high school":
                data[ir, ic] = 3
            elif cell.value == "Secondary school / high school completed":
                data[ir, ic] = 4
            elif cell.value == "Some university":
                data[ir, ic] = 5
            elif cell.value == "University completed" or cell.value == "Post-graduate" or cell.value == "Post-secondary qualifications, other than university":
                data[ir, ic] = 6
        ic += 1
    ic = 0
    ir += 1
    
#make the data of type int
data = data.astype(int)

### Node class

In [6]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

### Tree Class

In [7]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

In the next cell will process the data a bit to make it easier to use

In [8]:
temp = data[:, 1:]
rem = np.delete(temp,3,1)
Y = data[:, 4].reshape(-1,1)
Xtemp = data[:, 1:]
X = rem

The next cell splits the data into Training (60%), Validation (20%) and Testing (20%) sets

In [9]:
# first split the data into 60% Training and 40% Validation and Testing
X_train, X_testAndValidate, Y_train, Y_testAndValidate = train_test_split(X, Y, test_size=.4, random_state=42)

# then I split the 40% validation and testing data
X_validation, X_test, Y_validation, Y_test = train_test_split(X_testAndValidate, Y_testAndValidate, test_size=.5, random_state=42)

## Fitting the Model

The Decision Tree Classifier model will be created, fitted and and the final Decision tree Will be shown 

In [10]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train,Y_train)
classifier.print_tree()

X_2 <= 0 ? 0.004927990182773012
 left:X_5 <= 2 ? 0.013064762759395498
  left:X_4 <= -1 ? 0.021568047337278073
    left:0
    right:X_4 <= 2 ? 0.011692236990950178
        left:1
        right:1
  right:X_3 <= 1 ? 0.008967516069788795
    left:X_4 <= -1 ? 0.008092517921146958
        left:0
        right:1
    right:X_1 <= 1 ? 0.02770935960591131
        left:1
        right:1
 right:X_5 <= 5 ? 0.0042300714707079745
  left:X_0 <= 0 ? 0.0058762296054347685
    left:X_1 <= 1 ? 0.00856358648566441
        left:1
        right:1
    right:X_3 <= 1 ? 0.002553171064688553
        left:1
        right:1
  right:X_4 <= 2 ? 0.007260034918816827
    left:X_1 <= 3 ? 0.04392721960289536
        left:1
        right:0
    right:X_1 <= 1 ? 0.0037303556832658513
        left:1
        right:1


## Testing the model

The Accuracy and Accociated Confusion Matrix of the Training data is formed

In [11]:
Y_trainPred = classifier.predict(X_train) 
print("training accturacy:")
print(accuracy_score(Y_train, Y_trainPred),'\n')
print("confusion matrix")
print(confusion_matrix(Y_train, Y_trainPred))

training accturacy:
0.7257501744591766 

confusion matrix
[[  11  390]
 [   3 1029]]


The Accuracy and Accociated Confusion Matrix of the Validation data formed

In [12]:
Y_validPred = classifier.predict(X_validation) 
print("validation accuracy:")
print(accuracy_score(Y_validation, Y_validPred),'\n')
print("confusion matrix")
print(confusion_matrix(Y_validation, Y_validPred))

validation accuracy:
0.7322175732217573 

confusion matrix
[[  0 125]
 [  3 350]]


The Accuracy and Accociated Confusion Matrix of the Testing data is formed

In [13]:
Y_pred = classifier.predict(X_test) 
print("testing accturacy:")
print(accuracy_score(Y_test, Y_pred),'\n')
print("confusion matrix")
print(confusion_matrix(Y_test, Y_pred))

testing accturacy:
0.700836820083682 

confusion matrix
[[  3 141]
 [  2 332]]
